In [1]:
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import pandas as pd

import xml.etree.ElementTree as ET 


import os
import json 
import itertools
import xml.etree.ElementTree as ET

# nltk.download('stopwords')
# nltk.download('wordnet')

In [2]:

import numpy as np
import string
import nltk
import re 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re
class Textpreprocessor:


    def __init__(self, text):
        self.text = text
        self.processedText = text
        
    def covertToLower(self):
        self.processedText = self.processedText.lower()
        return self.processedText

    def removeTags(self):
        self.processedText = BeautifulSoup(self.processedText).get_text()
        return self.processedText
    
    def removeHyperlink(self):
        re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", self.processedText)
        return self.processedText

    def removePunctuation(self):
        self.processedText = self.processedText.translate((str.maketrans('','',string.punctuation)))
        return self.processedText
    
    def removeNumber(self):
        self.processedText = re.sub(r'[-+]?\d+', '', self.processedText)
        return self.processedText
    
    def removeStopWords(self):
        stop_words = set(stopwords.words('english'))
        self.processedText = [word for word in self.processedText if not word in stop_words]  
        return self.processedText
    
    def tokenize(self):
        self.processedText = word_tokenize(self.processedText)
        return self.processedText
    
    def stem(self):
        stemmer = PorterStemmer()
        self.processedText = [stemmer.stem(word) for word in self.processedText]
        return self.processedText
    
    def lemmatize(self):
        lemmatizer = WordNetLemmatizer()
        self.processedText = [lemmatizer.lemmatize(word) for word in self.processedText ]
        return self.processedText
    
    def getProcessedTokens(self):
        self.covertToLower()
        self.removeTags()
        self.removeHyperlink()
        self.removePunctuation()
        self.removeNumber()
        #--- string manipulation end ------

        self.tokenize()
        self.removeStopWords()
#         print(self.processedText)

        self.lemmatize()
        self.stem()
        return self.processedText

In [3]:
topicsFileList = []
# for topics txt 
# with open('Data/topics.txt') as f:
#     topicsList = f.read().split()
    
#     print(f.read())
# topicsList = [s + '.xml' for s in topicsList]

# for all 11 topics 
topicsList = os.listdir( 'Data/Training' )
topicsList.remove('3d_Printer.xml')
print(topicsList)



['Anime.xml', 'Arduino.xml', 'Astronomy.xml', 'Biology.xml', 'Chess.xml', 'Coffee.xml', 'Cooking.xml', 'Law.xml', 'Space.xml', 'Windows_Phone.xml', 'Wood_Working.xml']


## Gernerating Global Vocab List

In [4]:


    
files = os.listdir('Data/Training')
keep_rows = 500
globalVocab = dict()
for file in topicsList:   
    stringBuilder = []
    tree = ET.parse('Data/Training/' + file) 
    root = tree.getroot() 
    row = 0 
    
    for child in root:

        stringBuilder.append(child.attrib['Body'])
        
        row += 1
        if row == keep_rows:
            break

    processor = Textpreprocessor(' '.join(stringBuilder))
    tokens = processor.getProcessedTokens()
    
    for token in tokens:
        if token not in globalVocab:
            globalVocab[token] = 1
        else:
            globalVocab[token] += 1
            
globalVocabJson = json.dumps(globalVocab)

with open('globalVocabJson.json', 'w') as f:
    json.dump(globalVocabJson, f)

In [5]:
# loading global vocab
import ast
f = open('globalVocabJson.json','r') 

data = ast.literal_eval(json.load(f))
global_words_lst = list(data.keys())

In [6]:
len(global_words_lst)

19998

In [7]:
def generateBagOfWords(string):
    processor = Textpreprocessor(string)
    tokens = processor.getProcessedTokens()
    bag_of_word = dict()
    for word in global_words_lst:
        bag_of_word[word] = 0
    for token in tokens:
        if token in bag_of_word:
            bag_of_word[token] += 1
    vector = [*bag_of_word.values()]
    
    return vector 

In [8]:
# generating bag of words vector for training data
train_size, validation_size, test_size = 500,200,500




files = os.listdir('Data/Training')



X, Y = [], []
x_train, y_train, x_test, y_test, x_dev, y_dev = [], [], [], [], [], []
for class_no in range(len(topicsList)):
    file = topicsList[class_no]
    print(file)
    tree = ET.parse('Data/Training/' + file) 
    root = tree.getroot() 
    row = 1
    for child in root:
        vector = generateBagOfWords(child.attrib['Body'])
        if not np.array(vector).any():
            continue

        if row <= train_size:
            x_train.append(vector)
            y_train.append(class_no)
 
        elif row <= train_size + validation_size:
            x_dev.append(vector)
            y_dev.append(class_no)       
            
        elif row <= train_size + validation_size + test_size:
            x_test.append(vector)
            y_test.append(class_no)
        row += 1


Anime.xml
Arduino.xml
Astronomy.xml
Biology.xml
Chess.xml
Coffee.xml
Cooking.xml
Law.xml
Space.xml
Windows_Phone.xml
Wood_Working.xml


In [9]:
df = pd.DataFrame(data=np.column_stack((x_train,y_train)))
df.to_csv('vectorized_dataset_train.csv', index=False, header=None)

df = pd.DataFrame(data=np.column_stack((x_dev,y_dev)))
df.to_csv('vectorized_dataset_dev.csv', index=False, header=None)

df = pd.DataFrame(data=np.column_stack((x_test,y_test)))
df.to_csv('vectorized_dataset_test.csv', index=False, header=None)

In [10]:
# # df
# text = '<p> hello </br> this is inside tag </p> <html> jlksjdf < / html>'
# BeautifulSoup(text).get_text()


In [11]:
df = pd.read_csv('vectorized_dataset_test.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,19989,19990,19991,19992,19993,19994,19995,19996,19997,19998
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5495,0,0,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
5496,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,10
5497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
5498,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10


In [12]:
a = np.array([
    [1,2,3],
    [2,3,4],
    [5,6,7]
])
a[:2,:]

array([[1, 2, 3],
       [2, 3, 4]])